In [1]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jayeshmahapatra.github.io/2023/06/22/arcface.html")

docs = loader.load()

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/jayesh/miniconda3/envs/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model="zephyr")

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [6]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
response = retrieval_chain.invoke({"input": "What use arcface loss?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

ArcFace loss is commonly used in training face recognition systems due to its ability to achieve enhanced embedding separability compared to traditional softmax losses. By explicitly considering the angles between embeddings, ArcFace loss promotes angular separability of embeddings belonging to different classes while enforcing intra-class angular similarity. This results in more cleanly separated and compact embedding clusters during training, as demonstrated in visualizations provided by a GitHub repository called ArcFace-Embedding-Visualization. The use of ArcFace loss can be beneficial for tasks that require high degrees of inter-class separability, such as face recognition.


In [10]:
response

{'input': 'What use arcface loss?',
 'context': [Document(page_content="\\[L_3 = -log \\frac{e^{s \\cos (\\theta_{y_i} + m) }}{e^{s\\cos (\\theta_{y_i} + m)} + \\sum^{N}_{j=1,j\\neq y_i} e^{s \\cos \\theta_j}}\\]\n\nThis is the ArcFace Loss that is widely used in training face recognition systems.\n\nExperimental Results\nIn order to demonstrate the difference in class separation when using arcface loss vs classic softmax, I have created a github repository called ArcFace-Embedding-Visualization.\nThis repository contains contains code for visualizing how embeddings evolve when trained using ArcFace vs Softmax Loss, as shown below:\n\nVisualization of Embedding Separation across Training Epochs\n\n\n\n\n\n            ArcFace 3D embeddings during training.\n         \n\n\n\n            Softmax 3D embeddings during training.\n         \n\n\n\nAs we can see that the arcface loss results in embedding clusters that are more cleanly separated as well as are more compact than the embeddings t